In [2]:
# Tabla de datos

import pandas as pd
import os

dataBol=pd.read_excel(os.path.join('data','dataBolivia_indicadores.xlsx'))
dataBol.head()

,DEPARTAMENTO,PROVINCIA,TOTAL_VIVIENDAS,TAMAÑO_PROMEDIO_DEL_HOGAR,PORCENTAJE_ DE_VIVIENDAS_CON_ENERGIA_ELECTRICA,PERSONAS_CON_DIFICULTADES_PERMANENTES
0,CHUQUISACA,Oropeza,77376,3.62,87.08,10829
1,CHUQUISACA,Azurduy,5956,4.11,27.50,983
2,CHUQUISACA,Zudáñez,9116,4.21,48.96,1156
3,CHUQUISACA,Tomina,9217,3.77,46.23,1640
4,CHUQUISACA,Hernando Siles,8158,3.84,6367.00,1554
